In [4]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from sklearn.utils import shuffle
from transformers import T5TokenizerFast, TFT5ForConditionalGeneration
from progressbar import progressbar as pb

def essai(i):  
  A=tokenizer(inputbatch[i],padding='max_length',return_tensors='tf').input_ids
  B=model.generate(A)
  print(inputbatch[i])
  return print(tokenizer.decode(B[0]))

In [5]:
data = pd.read_csv('new_dataset.csv',index_col=0)
data['JOB']=data.JOB1+' '+data.JOB
data=data.drop(columns=['JOB1','GPE','TENSE'])
data=data[(data.txt.str.len()<80) & (data.JOB.notna()) & (data.PERSON!=data.ORG)]
# data=data[['txt','PERSON','NORP','JOB','ORG']]
data=shuffle(data).dropna()


df_input = data.iloc[:,1:]
df_label = data.iloc[:,0:1]

n = (len(df_input)//100)*100
print(n)
print(len(df_input))

1700
1706


In [6]:
model = TFT5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5TokenizerFast.from_pretrained("t5-base")
optimizer = tf.keras.optimizers.Adam()

2021-08-10 08:55:31.873376: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-10 08:55:31.873902: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-10 08:55:31.874129: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (HAL-9000): /proc/driver/nvidia/version does not exist
2021-08-10 08:55:31.878320: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-10 08:55:31.932626: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but

In [7]:
batch_size=32
num_of_batches=math.floor(n/batch_size)

label=[df_label.values[i][0] for i in range (len(data))]
label_encoded=tokenizer.batch_encode_plus(label,padding=True,max_length=900,return_tensors='tf')["input_ids"]

inputbatch=['|'.join(df_input.iloc[i].dropna().values.tolist()) for i in range (len(data))]
input_encoded=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=900,return_tensors='tf')["input_ids"]

In [8]:
epoch=2
los=[]
for k in range(epoch):

    running_loss=0
    
    
    for i in pb(range(num_of_batches)):
        input=input_encoded[i*batch_size:i*batch_size+batch_size]
        output=label_encoded[i*batch_size:i*batch_size+batch_size]
        
        #Entrainement sur le batch
        with tf.GradientTape() as tape:
            outputs = model(input_ids=input, labels=output)
            # Compute the loss value for this batch.
            loss_value = sum(outputs.loss)/len(outputs.loss)
            running_loss+=loss_value
        # Actualisation des poids
        gradients = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))




100% (53 of 53) |########################| Elapsed Time: 1:21:23 Time:  1:21:23
100% (53 of 53) |########################| Elapsed Time: 1:01:30 Time:  1:01:30


In [26]:
A=tokenizer("Teddy Riner|french|judo player",padding='max_length',return_tensors='tf').input_ids
B=model.generate(A)
print(tokenizer.decode(B[0]))

<pad> Teddy Riner is a French former judo player</s>


In [10]:
model.save_pretrained("../model")